# Chapter 1: L3 Technologies  
## Admin Distance 
* Belivability of a routing advertisement 

| AD | Routing Source |
|---| ---|
|0 | Connected |
|1 | Static (default) |
|20| eBGP |
|90| Internal EIGRP (D) |
|110 | OSPF |
|115 | IS-IS |
|120 | RIP |
|170 | External EIGRP (D EX) |
|200 | iBGP|
|255 | Static default over DHCP |

* Verify by ip Routing table `sh ip route` each prefix has `[AD/Metric]` values attached
* To alter AD of static route using __Floating Static Route__: `ip route DEST_IP NH_IP DIST`

## Redistribution basics
### Needs of redistribution 
* Single AS runnning multiple IGP 
* IGP communicated with BGP and vise-versa 

### Challenges 
* Each routing protocol has its own metric. While redistributed each RP uses a default __Seed Metric__. 

| Routing Protocol | Seed Metric |
|--|--|
| RIP | $\infty$ |
| EIGRP | $\infty$ |
|OSPF | 20 (1 for BGP) |
| BGP | Uses IGP Metric |


## Mutual Route-Redistribution 
![](Figs/mutual_redis.png)

Router R2 runs both OSPF and EIGRP with appropreate settings. However, R1 is not aware about R3 routes and vice-versa. In __Mutual Route-Redistribution__ R2 will mutually redistribute routes learnt from R1 and R3 among each other.
```
R2#sh ip route ospf 
     192.168.12.0/32 is subnetted, 1 subnets
O       192.168.12.1 [110/2] via 12.0.0.1, 00:01:41, FastEthernet0/0
     192.168.11.0/32 is subnetted, 1 subnets
O       192.168.11.1 [110/2] via 12.0.0.1, 00:01:41, FastEthernet0/0
  
R2#sh ip route eigrp 
D    192.168.21.0/24 [90/156160] via 23.0.0.3, 00:00:34, FastEthernet0/1
D    192.168.22.0/24 [90/156160] via 23.0.0.3, 00:00:34, FastEthernet0/1
```

__Syntax__
```
conf t
    router TARGET_RP
        redistribute SOURCE_RP 
end 
```

* Redistribution EIGRP AS 1 into OSPF in classless mode with custom metric 40

```
!r2
router ospf 1
    redistribute eigrp 1 subnets metric 40
    
!r1 
sh ip route ospf
     23.0.0.0/24 is subnetted, 1 subnets
O E2    23.0.0.0 [110/40] via 12.0.0.2, 00:00:28, FastEthernet0/0
O E2 192.168.21.0/24 [110/40] via 12.0.0.2, 00:00:28, FastEthernet0/0
O E2 192.168.22.0/24 [110/40] via 12.0.0.2, 00:00:28, FastEthernet0/0
```

* Redistribute OSPF into EIGRP AS 1 with appropreate metric. Check the metric=170 for DEX routes.

```
!r2
router eigrp 1
    redistribute ospf 1 metric 1000000 10 255 1 1500
    
!r3
R3#sh ip route eigrp 
     192.168.12.0/32 is subnetted, 1 subnets
D EX    192.168.12.1 [170/30720] via 23.0.0.2, 00:00:08, FastEthernet0/0
     192.168.11.0/32 is subnetted, 1 subnets
D EX    192.168.11.1 [170/30720] via 23.0.0.2, 00:00:08, FastEthernet0/0
     12.0.0.0/24 is subnetted, 1 subnets
D EX    12.0.0.0 [170/30720] via 23.0.0.2, 00:00:08, FastEthernet0/0
```

## Altering seed metric 
* __Option 1: gloabally for a target routing proto__ where all redistributed routes will have identical metric, regardless of their source. Make sure put the configuration ont he distributing router.
```
router ospf|eigrp ASN
    default-metric [ARG1, ARG2,...]
```
* __Option 2: in `redistribute` command__ where the metric is specified int the `redistribute` command with `metric` followed by RP-specific parameters.
* __Option 3: using `route-map`__ where metric alteration is dictated by a routing policy. 

```
conf t
    route-map MAP_REDIST !create a RMAP for any match 
        set metric 1000000 1 255 1 1500
    exit
    router eigrp 1  !apply the RMAP to the EIGRP redist  
        redist ospf 1 route-map MAP_REDIST
end 
```
* E1 and E2 route in OSPF: OSPF redistributes with metric-type 2 (E2) by default that does not change while hopping over IGP domain, to make it use E1. using `redist SRC_RP subnets metric VAL metric-type 1|2` command. 

# Route Filtering 
Filter or choose which routes are permitted/denied to get redistributed. 

__TASK__: Allow only 192.168.11.0/24 and 192.168.21.0/24 routes to be redistributed. 

```
conf t
    !create an ACL of permitted routes 
    ip access-list stand ROUTE_FILTER_LIST
        10 permit 192.168.11.0 0.0.0.255
        20 permit 192.168.21.0 0.0.0.255
    exit
    
    !create a route-map from the ACL 
    route-map ROUTE_FILTER_MAP permit 10
        match ip address ROUTE_FILTER_LIST   ! if matches the ACL prefixes
            set metric 1000000 10 255 1 1500 ! set the metric also 
    exit
    route-map ROUTE_FILTER_MAP deny 20       ! deny for others  
    exit
    
    ! apply route filtering on the RPs
    router ospf 1 
        redist eigrp 1 subnets route-map ROUTE_FILTER_MAP
    router eigrp 1
        redist ospf 1 route-map ROUTE_FILTER_MAP
end 
```


## Routing Loop
There are two problems that come with redistribution 
* __Routing Loop:__ Routing updates flooded in loop within the routing domain 
    * To prevent any avdertisement to come back the source routing domain (AS) __Route-Tagging__ is used. Each redistributed routes is tagged by the boundary router and it also prevents routes with same tag to enter (__NO-RETURN__) 
    * For example, while redistributing routes from OSPF to EIGRP domain, R2 tags them with __TAG=10__ and R4 is going to deny any tag=10 from entering. 
* __Suboptimal Routing:__ Routers chooses non-best path for preferring a slow link with a higher AD routing protocol over a fast link with a lower AD RP 

![](Figs/routing_loop.png)

In the figure, R2 recived 14.0.0.0/24 via R1 over internal OSPF (AD=110) and via R3 over external EIGRP (AD=170). therefore, R2 chooses 12.0.0.1 as a next hop than 23.0.0.3. You can verify the entry of 14.0.0.0/24 in both OSPF LSDB and EIGRP topology table. However, the routing table will always choose the best route. 

```
!R2
sh ip eigrp topology | sec 14.0.0.0
P 14.0.0.0/24, 1 successors, FD is 5120
        via Redistributed (5120/0)

sh ip os database | in 14.0.0.1    
14.0.0.1        192.168.12.1    1468        0x80000003 0x0025F0
```
Now if you shut the F0/0 interface of R2, it should reconverge by choosing a path via R3.

```
!r2
!after shutting down f0/0
trace 14.0.0.1

Type escape sequence to abort.
Tracing the route to 14.0.0.1

  1 23.0.0.3 60 msec 32 msec 28 msec
  2 34.0.0.4 64 msec 60 msec 64 msec
  3 14.0.0.1 80 msec 72 msec 108 msec

!
!after no-shutting f0/0
trace 14.0.0.1

Type escape sequence to abort.
Tracing the route to 14.0.0.1

  1 12.0.0.1 32 msec 28 msec 28 msec
```

### Applying Route-Tag
Apply route-tag there must te two route-map. One will tag the routes (TAG_20) another will deny the routes with tag=20 and allow others (DENY_TAG_20). Finally apply the route-maps to the appropreate direction, i.e. tagging 20 while redistributing into EIGRP from OSPF and denying tag 20 while redistribution back to OSPF from EIGRP. 

```
!R2
conf t
    route-map TAG_20 permit  !set tag=20
        set tag 20
    exit
    
    route-map DENY_TAG_20 deny 10  ! deny tag=20
        match tag 20
    route-map DENY_TAG_20 permit 20 ! permit any
    exit
    
    !applying route-tags into redistribution 
    router eigrp 1
        redist ospf 1 route-map TAG_20 metric 1000000 10 255 1 1500
    router ospf 1
        redist eigrp 1 subnets route-map DENY_TAG_20
end 
``` 

to verify the tagged routes, check the EIGRP topology table and check only redistributed routes from OSPF is tagged with 20.

```
!r2
sh ip eigrp topology | in tag

P 12.0.0.0/24, 1 successors, FD is 5120, tag is 20
P 14.0.0.0/24, 1 successors, FD is 5120, tag is 20
P 192.168.11.1/32, 1 successors, FD is 5120, tag is 20
P 192.168.12.1/32, 1 successors, FD is 5120, tag is 20
```
do the same configuration in R4.

## Redistribution on IPv6

* IPv6 redistribution does not include any Connected routes.
* Explicit command __`include-connected`__ is needed to include connected routes (Logic: Only foreign routes are feasible to be redistributed). 
* The subnet option is implicit for OSPFv3 as IPv6 has no concept of subnets.

```
conf t
    ipv6 router eigrp 1
        redistribute ospf 1 metric 1000000 10 255 1 1500 include-connected
    ipv6 router ospf 1
        redistribute eigrp 1 include-connected 
end 
```

## Troubleshooting route-redistribution 

* Does the border router run both the protocol with proper AS/PID
* Check the routing table if both routes are visible in R2
* Check if the redistribution is appltied (check running confing router section)
* Check if the default metric set for non-ospf protocols 
* if default-metric is not set then put metric command in reditribute statement or in a route-map. 

## Policy Based Routing (PBR)

* Overrides the router's default forwarding decission using custom logic 
* Uses ACL to match a prefixes and a Route-Map to match and take action defined in the ACL
* Common operations 

| Operation | Attributes |
|---|---|
|__Match__ | IP Address |
|| Range of packet Length |
|__Set__ | Next hop IP Address |
|| Default Next-hop IP |
|| Egress Interface |
|| Default interface |

* PBR is not applied by default on any traffic that is originated from the subjected router, to make it happen a varient of PBR (Local-PBR) is to be used.

![](Figs/pbr.png)

### Task: 
* Configure R1 in such a way that any traffic going to 192.168.21.0 and 192.168.22.0 network must take R2 and R4 as a next hop respectively.
* __Steps__:
    * Create a ACLs to match the routes 
    * Create a single route-map with two match statements, each matching individual ACL and set their appropreate actions.
    * Apply the policy to the ingress interface with __`ip policy route-map MAP_NAME`__ command
* __Confiuration__:

```
!r1
    conf t
        ! ACL for matching
        ip access-list ext PBR_21
            10 permit ip any 192.168.21.0 0.0.0.255
        ip access-list ext PBR_22
            10 permit ip any 192.168.22.0 0.0.0.255
         
        ! route-map for Policy definition
        route-map EXIT-POLICY permit 10
            match ip address PBR_21
                set ip next-hop 12.0.0.2 
        exit
        route-map EXIT-POLICY permit 20
            match ip address PBR_22
                set ip next-hop 14.0.0.4 
        exit
        
        ! apply policy to ingress interface
        int e1/0
            ip policy route-map EXIT-POLICY
```
* __Verification__:

```
PC1> trace 192.168.21.1
trace to 192.168.21.1, 8 hops max, press Ctrl+C to stop
 1   192.168.13.1   17.229 ms  14.715 ms  14.755 ms
 2   12.0.0.2   47.834 ms  45.625 ms  47.074 ms
 3   *23.0.0.3   77.558 ms (ICMP type:3, code:3, Destination port unreachable)

PC1> trace 192.168.22.1
trace to 192.168.22.1, 8 hops max, press Ctrl+C to stop
 1   192.168.13.1   16.773 ms  15.319 ms  14.704 ms
 2   14.0.0.4   46.325 ms  45.992 ms  46.096 ms
 3   *34.0.0.3   75.496 ms (ICMP type:3, code:3, Destination port unreachable)

```

## Virtual Routing & Forwarding (VFR)

![](Figs/vrf.png)
* Creates virtual routing instances on top of a physical router 
* ISP needs to maintain individual routing instances for individual customer to seperate overlapping routes, this leverages multi-tanancy.
* The virtual routing instances creates logical Routing table along with vRIB and vFIB along with the global routing table with RIB and FIB.
* __VRF Lite__ is a special varient that does not include MPLS 
* VRFs run in isolation by default, however routes can be injected among them explicitly by __Route-Leaking__
* __Config__:
```
conf t
    ! creating VRF
    vrf definition TENANT_A
        ip address-family ipv4
    vrf definition TENANT_B
        ip address-family ipv4
    
    ! registering interfaces into VRF
    ! notice TENANT_A and B has overlapping address
    int e1/0
        vrf forwarding TENANT_A
        ip add 192.168.101.1 255.255.255.0
        no sh
    int e1/1
        vrf forwarding TENANT_A
        ip add 192.168.102.1 255.255.255.0
        no sh
    int e1/2
        vrf forwarding TENANT_B
        ip add 192.168.101.1 255.255.255.0
        no sh
    int e1/3
        vrf forwarding TENANT_B
        ip add 192.168.102.1 255.255.255.0
        no sh
 end 
```

# Chapter 2 Troubleshooting Routing Protocols 

## Enhanced Interior Gateway Routing Protocol

### EIGRP Fundamentals

* __EIGRP Overview__
    * Fast Convergence 
    * Scalable 
    * Load Balancing over unequal cost links using _Varience_ 
    * Classless (VLSM) support
    * Communicate via multicast: `224.0.0.10` 
    * Open standard since 2010 
* __EIGRP Main components__
    * RTP based reliable transport of routing advertisement 
    * DUAL algorithm to find the best path (Feasible distance based) 
    * Protocol dependent modules (supports IPv4, IPX, Appletalk and IPv6)
    * EIGRP Data structure 
        * Neighbour table : `sh ip eigrp nei`
        * Topology table  : `sh ip eigrp topology [all-links]`
        * Interface Table : `sh ip eigrp interface`

### EIGRP Timers

* Hello interval : 5 Sec (how often keep-alive should be sent).
* Hold-Down interval : 15 Sec (Neighbour, consider me dead if you dont hear from me for 15 sec). This is different than OSPF dead timer. See Hold column using `sh ip eigrp nei`.

### EIGRP Metric Calculation

* Composite Metric: Bandwith, Delay, Reliability, Load and MTU 
* By default: Bandwidth and Delay is used $(k_1=k_3=1 ; k_2=k_4=k_5=0)$
* Metric Formula: $metric = 256\bigg[\bigg(k_1 \frac{10^7}{B_{min}} + \frac{k_2\frac{10^7}{B_{min}}}{256-L}+k_3\sum{D}\bigg)+\frac{k_5}{k_4+R}\bigg]$

### EIGRP Path Selection
![](Figs/eigrp_rd.png)

* __Reported Distance (RD)__: Distance to a network reported by a neighbour 
* __Feasible Distance (FD)__: Mininum of reported distance 
* __Successesor (S)__: Next-hop long the primary path. 
* __Feasible Successor (FS)__: A neighbour who is not a successor but obays the _Feasibility Criteria(FC)_.


### Feasibility Criteria (FC)  
Rule : For a neighbour to become a feasible-successor (FS) its $RD \lt FD$ for the subjected router.
* This criteria is there as EIGRP is not link-state routing protocol rather an advanced distance vector routing protocol therefore it does not know about the complete topology map. 
* However, there is a caviat in the FC. Sometimes, FC rejects some of the valid routes to be feasible path. 
* A valid path, rejected by FC comes live during query process

### EIGRP Query process
![](Figs/eigrp_qry.png)

* When the a router looses connectivity to a network and any feasible path is not found 
* it generates a Query packet to its neighbour and the process continues recursively untill the a router with no neighbour is reached. 
* The final downstream router sends a reply packet to upstream routers and the process recursively ends continues untill the reply reaches the source router. 
* The source router needs to wait untill the query process finishes. marking the down route into __Active__ state. 
* This raised a problem called __Stuck in Active__. If any REPLY packet gets lost during the transit from a down stream router, the next upstream route waits for 3 minuits before it declaire the entire downstream dead. 
* This problem is solved by __SIA Packet__. If no reply comes untill 90 seconds, the source sends a __SIA Query__ asking if the neighbour is down or it is waiting to get the reply. The neighbur confirms its livelyness by sending a __SIA Reply__.  

### Basic Config

```
conf t
    router eigrp 100             ! asn=100
        no auto-summa            ! turning off auto-summary 
        passive default          ! turn eigrp adv off from all ifaces

        network 0.0.0.0          ! running eigrp on all active interface
        neteork NET_ID WC_MASK   ! activate interfaces with matching NET_ID
        network INT_IP 0.0.0.0   ! Activate specific interface (Recomanded)

        no passive IFACE         ! activating eigrp on desired ifaces only
end  
```

### Stub Routing 

* A __Stub Router__ is a terminal router where Query is not sent. 
* A Stub router only advertises _Directly connected_ and _Summary_ Routes.
* Use __`eigrp stub [connected | static | summary | leak-map | recive-only | redistributed ]`__ on global routing mode to make a router stub, this will reset the neighbourhip.
    * default option is connected and summary 
    * __Connected | Static | Redist | Summary__ only 
    * __Leak-Map__: If certain routes to be advertised 
        * Create an ACL to match those desiared routes
        * Create a Route-Map to refer the ACL for matchting the IP Addresses
        * Refer the Route-Map in the Leak-Map
    * __Receive-Only__: stub router will supress any advertisement  
    
* Verify from its neighbours using `sh ip eigrp neighbour detail` commnad.

### Load Balancing 

* EIGRP unlike OSPF allows __Unequal Cost Load Balancing__ 
* It uses the __Varience__ value for this. Any __feasible successor routes__ within the range of __$[FD,var\times FD]$__ is eligble for unequal cost LB. 
* use the __`varience VAL`__ under router config mode, to set the varience. 

### Route-Summarization 

* Reduce the size of routing table 
* Make sure the the IP range is not discontigious 
* Unline OSPF every router is a potential summary point 
* Reduces the Query Scope: An EIGRP speaker does not propagate Querry message if the qurried route is not an exact match, summary routes never be an exact match, therefore the subjected routers dont propagate, which reduces the scope.  
* May cause suboptimal routing: Make sure the summary does not match any specific route elsewhere in the network. 
*  __Manual Summary__: Use command __`ip summary-address eigrp ASN SUMM_ADDR MASK`__ in interface config mode to advertise any summary. 
* __Auto Summary__: use command __`auto summary`__  in router config mode to perform class-full summarization. However, make sure the same classful net is not used anywhere. Therefor, auto-summary is turned off in EIGRP. 

### EIGRP on IPv6

* Next hot is neighbour's link local address
* No auto summary 
* Neighbours dont need to be in a same subnet 
* Sends update to `FF02::A` (FF=IPv6 MC, 02=Scope, A=10)
* Network command is not there, rather advertisement is configured in interface level 

```
conf t
    ipv6 unicast-routing
    ipv6 router eigrp ASN
        
    int f0/0
        ipv6 eigrp ASN
end 

```

### EIGRP Named mode

configuration of EIGRP needs different modes for different types of configuration, that is sometimes confusing. Therefore, the __Named-Mode__ config exists. 

* Different hierarchy-level of configuration 
    * __Address-Family__
        * Router ID
        * Stub Router 
    * __Address-family-interface__
        * Passive Interface 
        * Timers
    * __Address-family-topology__
        * Varience
        * Redistribution
* Named-mode EIGRP is __backward compatible__ to EIGRP v4 and v6. 

![](figs/eigrp_named.png)
        
Let the routers are pre-configured with classic EIGRP v4. Now, lets turn off EIGRP on R2 that will break the neighbourship first. Then, if it is configured in named mode, despite R1 and R3 are configured with classic mode R2 will sync the topology as Named mode is backward compatible. <br>
EIGRP named mode was designed for advertising beyond routes, e.g. __`Service-Family`__ can advertise phone number prefixes in a VoIP communication network. 

```
conf t
    router eigrp ASN1       ! Entering into Named mode configuration
        address-family ipv4 unicast autonomous-system 1
            topology base 
                no auto-summary
                exit
            
            af-interface default 
                passive interface 
            
            net 0.0.0.0 
            
            af-interface f0/0 
                no passive-int
            af-interface f0/1
                no passive-int 
end 
```

If you check the protocol status of eigrp, it shows a new k-value __K6__ which comes from the EIGRP wide metric, used in Named mode metric calculation.

```
sh ip protocols | inc weight
    Metric weight K1=1, K2=0, K3=1, K4=0, K5=0 K6=0
```

### EIGRP Authentication

* Primary use of authetication is to prevent a malecious user to infect the routing table by only allowing authenticated routers to exchange routes. Authentication is done by a __Local-Key__ 
* Types of authentication: Plain-text and Hashed key ; Plain-text key is 
* Routing Update + local key = digest 
    * MD-5
    * SHA
* To give better security, Cisco changes the keys periodically using __Key-Chain__. 
* Make sure to overlap the timeframe if multiple key is configured. 

```
conf t
    key chain TEST_CHAIN
        key 1
            key-string PRIMARY_KEY   ! string must match on the pairing routers
            accept-lifetime 00:00:00 june 1 2020 july 1 2020 
            send-lifetime   00:00:00 jue 1 2020 july 1 2020
        
        key 2                         ! optional                          
            key-string SECONDARY_KEY  ! string must  
            accept-lifetime 00:00:00 june 30 2020 infinite 
            send-lifetime   00:00:00 june 30 2020 infinite
    exit 
    do sh key   ! to verify the key-chain 
    
    !v4 configuration 
    int f0/0
        ip auth mode eigrp 1 md5             !which hash to use 
        ip auth key-chain eigrp 1 TEST_CHAIN !which keychain to use 
     
    !v6 configuration 
    int f0/1
        ipv6 auth mode eigrp 1 md5 
        ipv6 auth key-chain eigrp 1 TEST_CHAIN 
end 
```


## Open Shortest Path First (OSPF)

### OSPF Fundamentals 

* OSPF has always been an Open Standard 
* Link State Routing Protocol (LSRP) : Maps the entire network topology in to Link-State Database (LSDB), _Think of Zig-Saw Puzzle_ 
* Finds shortest path by running Dijkstra's SPF algorithm on top of LSDB 
* __Basic Terminologies__
    * Hello : __Hello Protocol__ for Neighbour Discovery + Reachability + DR Election 
    * LSA : __Link State Advertisement__ , Information that each router sends about their reachability to contruct LSDB.  
    * LSU : __Link State Update__, A packet that carries LSAs 
    * LSR : __Link State Request__, Requesting for a missing peice i.e. a specific LSA 
    * LSAck : __Link State Ack__, Acknowledging an recived LSR
* __Adjacency and Neighbourship__
    * Neighbours : Routers on same segment sharing hello messages, with the following requirement.
        * Matching Areas, Authentication, Subnet, Timers, Stub-Flag, MTU ( otherwise Stuck in EXSTART/EXCHANGE state). 
    * Adjacent : Neighbours  + Exchange Link state information (LSU & DD) 
    * Why can't we make a full-adjacency? 
        * Number of adjacency = $O(n^2)$ 
        * Designate a router (DR) as a hub and a backup (BDR) to make full adjacency to all other (DRO) routers, making the topology of $O(n)$ order. 
        * Multicast Groups: 
            * `224.0.0.5 / FF02::5` : All OSPF Routers 
            * `224.0.0.6 / FF02::6` : All DR 
* __DR/BDR__ Election 
    * Every router first pretends to be a BDR
    * __Priority__ value is carried by Hello Packet, Heighest priority wins. To change, __`ip ospf priority NUM`__ at interface config mode.
    * A Beefy router / Hub must always be set a DR
    * __Prio = 0__ means not participating into election process
    * If Tie with Priority use __Router ID__ 
        * Auto: Highest loopback $\rightarrow$ Highest active interface IP
        * Manual: __`router-id ID`__ in router config mode
    * If no interface IP is setup, OSPF will not start. 

* __OSPF Areas__
    * To limit the flooding of LSU and the LSDB size 
    * Simplify troubleshooting 
    * Cisco recomandation not to exceed 50 routers under a single area. 
    * Area 0 is called Backbone area, all router must be adjacent to Area 0. 
    * A router that has at least one interface connected to Area 0 is called Area Border Router (ABR) 
    * A Rpouter that has at least one interface conneted to a Non-OSPF domain is called Autonomous System Border Router (ASBR) 

### OSPF Timers 

* __Hello Timer__: The interval at which, router sends hello messages out of an OSPF-enabled interface. Change with __`ip os hello-interval SEC`__ on the interface config mode.
* __Dead timer__: The interval that an OSPF enable interface waits to recive a hello message from a neighbour before considering that neighbour to be down. 4 time of Hello. Change with __`ip ospf dead-interval SEC`__ on the interface config mode.
* __Wait timer__: Waiting for DR/BDR election.  
* __Retransmit timer__ : How long  a router would wait before re-transmitting a LSU if the ACK not received.  Change with __`ip ospf retransmit-interval SEC`__ on the interface config mode.
* Different Network types have different default timers

### OSPF Cost Calculation 

* $Cost = \frac{B_{ref}}{B_{int}} = \frac{10^7 bps}{B_{int}}$ 
* Any interface over 100 Mbps is of cost = 1. To solve this change the Reference Bandwith using __`auto-cost reference-bandwidt MBPS`__ in router OSPF mode. 

## OSPF Network Types

| Network Type | Encapsulation | DR/BDR | Hello Timer | Neighbourship |
| --- | --- | --- | --- | --- |
|Broadcast | Ethernet | Yes | 10 sec | Dynamic |
|Point-to-point| Serial (HDLC/PPP) no FR | No | 10 | Dynamic | 
|Non-Broadcast | Serial with FR | Yes | 30 | Static |
|Point-to-multipoint | Serial with FR | No | 30 | Static | 

__Some Valid Combination of Network types__

 |  | BC | NBMA | P2P | P2MP | 
 |-|-|-|-|-|
 |__BC__| Yes | Yes (timer adjust) | No | No |
 |__NBMA__|  | Yes | No | No |
 |__P2P__| |  | yes | Yes | 
 |__P2MP__| |  | | yes |

### LSA & Area Types

* Hold the unit information to build te LSDB 
* LSDB is formed per Area
* __LSA Types__
    * __Type 1__: A __Router LSA__ is created by each router and contains information about that router's directly conneted netoworks. Number of LSA-1 flooded within an area equals to the number of routers attached to that area. 
    * __Type 2__: A __Network LSA__ is created for each transit network within an area on which a DR is elected. Number of LSA-2 flooded within an area is equals to number of network segments (DRs) within that area. 
    * __Type 3__: A __Summary LSA__ is sent from one area to another by an __ABR__ to advertise Inter-Area routes. The summary is not on the prefixes, rather it is called summary as the ABR gives a summarised map to the O-IA routes instead of a detailed one. 
    * __Type 4__: A __Summary ASBR LSA__ is created by an ABR to tell memebers of an area how to reach and ASBR. 
    * __Type 5__: An __AS External LSA__ is created by an ASBR to advertise networks in a different AS. 
    * __Type 7__: An __NSSA LSA__ is sent by a ASBR into an NSSA to advertise external routes.  
* __Area Types__
    * __Backbone Area__ : Area = 0 / 0.0.0.0 is called the Backbone area. Every OSPF router must have a direct or indirect (through Virtual Link) connection to the Backbone Area.
    * __Stub Area__ : When an ASBR floods Type 5 LSA, an ABR must relay them to the other memeber routers plus an Type-4 LSA to create reachability to the ASBR. In case an area does-not want that granular information of external prefixes, it can create a default route for all unknonwn prefixes pointing to the ABR as a gateway. In that case Type-5 and Type-4 LSA will not flood into that area, rather it ABR will flood a Type-3 LSA to maintian a default route. This area is called __Stub Area__
    * __Totally Stubby Area__ : If a Stub area does not need to maintain individual O-IA routes, it can summarise the LSDB even further by only allowing a single Type-3 to flood to have a default route via the ABR. Then the area is called __Totally Stubby__. 
    * __Not-So-Stubby Area (NSSA)__ : If a Stubby area has a ASBR connected, then the ASBR would inject Type-5 LSA to advertise external routes. However, type-5 LSA does not get flooded in Stub area. Therefore, the ASBR generates a Type-7 LSA that flows through the NSSA and the ABR converts it into a Type-5 LSA. 
    * __Totally NSSA__ : 
    
![](Figs/ospf_areas.png)

# Chapter 3 : Infrastructure Security and VPN 

## VPN

### MPLS Operations 

![](Figs/mpls_lsp.png)

* __Multiprotocol Label Switching__
    * Uses labels to make packet forwarding decisions
    * Flexible and efficient 
    * Supports many different services 
* Label switch router: Performs routing based on Labes 
    * In a typical router Control plane contains RIB and Data plane contains FIB
    * in LSR the control plane is LIB and the Data Plane is LFIB with a __Label distribution protocol LDP__. Any unlabeled packet is handled by FIB. 
    * Inside the MPLS domain a __Label Switched Path (LSP)__ is calculated to forward packets. Theoretically routers may chose different paths during a round-trip, however, the Unicast Routing protocols prevent it. 

__MPLS Header__ is a shim header, i.e. it is stuffed between the L2 and L3 headers. 

![](Figs/mpls_header.png)

* 20 bits is the label 
* 3 bits __EXP__ is for QoS
* 1 bit __BoS__ is the Bottom of stack bits indicates if the label is the last label. 
* 8 bits __TTL__ to prevent loops 

### MPLS VPN

* Forward packets based on labels insead of IP 
* Combines the best of Overlay and P2P VPN model 
* ISP Core routers are not aware about the Customer routes, BGP-Free Core. 
* __Steps to implement MPLS-VPN__
    * Configure __IGP__ inside the SP core 
    * Configure __MPLS LDP__ inside the SP Core 
    * Create __VRF, RD, RT__. 
    * Configure __VPN-v4 peering__ between bith the PE routers 
    * Configure __PE-CE Routing__ (Static/ def/ IGP/ BGP) 
    * COnfigure __Redistribution__ on PE routers
